# Experiment: Explore Water Management Strategies

Water Management is a complex topic. It involves many different actors, from the government to the private sector, and many different strategies, from water pricing to water reuse. In this experiment, we will explore the different strategies that countries have adopted to manage their water resources.

Especially, Waste Water Treatment and Reuse are important topics.

In [91]:
import sys
import os
sys.path.insert(1, os.path.abspath(os.getcwd()[:-3]))

from src.aquastat_utils import *

# Import AQUASTAT
aquastat_df = get_aquastat()

Getting AQUASTAT dataframe from fao_aquastat.csv ...
/Users/josef/Sync/Dokumente/SCHULE/Master/UT/Vorlesungen/Sem2/Data_Literacy/Paper/src/../dat/fao_aquastat.csv already exists.
Renaming countries ...


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from tueplots import bundles

plt.rcParams.update(bundles.icml2022())
plt.rcParams.update({"figure.dpi": 200})

# colormap for spatial plot
rb = LinearSegmentedColormap.from_list(
    "rb",
    [[0, 0, 153 / 255], [50 / 255, 150 / 255, 255 / 255], [1, 1, 1], [255 / 255, 102 / 255, 0], [153 / 255, 0, 0]],
    N=500
)


In [ ]:
# Filter interesting variables
filter_for = ['waste', 'reuse', 'treatment']  # variables interesting for waste water treatment
filter_for = ['waste']

filtered_columns = [column for column in aquastat_df.columns if any([x in column.lower() for x in filter_for])]
filtered_columns

We now have a list of variables related to waste water and treatment. Let's explore them.

In [ ]:
# Get for each country the number of municipal wastewater treatment facilities

# Create a new dataframe with only the columns we are interested in
interesting_var = 'Number of municipal wastewater treatment facilities'

waste_df = aquastat_df[['Country', 'Year', interesting_var]]
waste_df = waste_df.dropna()  # Drop rows with missing values
waste_df = waste_df.sort_values(by=interesting_var, ascending=False)  # Sort by number of facilities

waste_df

The total number is not very interesting to us so lets look at facilities per capita.
We dont want to look at to small numbers so per 1000 people seems appropriate

In [ ]:
'''Get facilities per n citizens'''
waste_df['Total population'] = aquastat_df['Total population']
n = 1000
waste_df[f'{interesting_var} per {n} citizens'] = (waste_df[interesting_var] / waste_df['Total population']) * n
waste_df

Plot the development of the number of facilities over time

In [ ]:
'''Just for testing'''
sns.scatterplot(data=waste_df[waste_df['Country'] == 'Canada'], x='Year',
                y='Number of municipal wastewater treatment facilities')
plt.show()

In [ ]:
import numpy as np
import geopandas as gpd
from src.utils import to_dat_path, save_fig


def get_growth_rate(series, log_scale=False):
    """
    Calculate the relative growth rate of a series.
    !! It only looks at the first and last value in series !!
    :param series: series to calculate relative growth rate for.
    :return: Relative growth rate.
    """
    y = series.values
    rate = ((y[-1] - y[0]) / y[0]) * 100
    if log_scale:
        rate = np.log(rate)

    return rate


def plot_growth_rates(
        data: pd.DataFrame,
        variable: str,
        cmap: str = 'coolwarm_r',
        title_var: str = None,
        log_scale: bool = False
):
    """
    Plot relative growth rates for a variable on a world map.
    :param data: Dataframe containing countries, years and variables to plot.
    :param variable: Variable to plot.
    :param cmap: Preferred colormap for plotting. The Standard is 'coolwarm_r'.
    :param title_var: Preferred form of variable in title.
    :param log_scale: Whether to use a log scale for the growth rates.
    :return: Nothing
    """

    font_size = 15

    '''Get Rates'''
    # Pivot the DataFrame to have years as the index and countries as columns
    df_pivot = data.pivot(index='Year', columns='Country', values=variable).dropna()
    # Apply the function to calculate growth rate for each country
    rates = df_pivot.apply(get_growth_rate, log_scale=log_scale)
    # Convert the results to a DataFrame
    rates_df = rates.reset_index(name='Relative growth rate')

    # Get map
    world = gpd.read_file(to_dat_path(file_path='naturalearth/ne_110m_admin_0_countries.shx'), engine="pyogrio")
    # Join Data to map
    merged = world.set_index('SOVEREIGNT').join(rates_df.set_index('Country'))
    vmax = max(abs(merged['Relative growth rate'].min()), merged['Relative growth rate'].min())

    # Plotting
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    merged.plot(
        column='Relative growth rate',
        ax=ax, legend=True,
        missing_kwds={"color": "darkgrey", "label": "No Data", },
        cmap=cmap, vmin=-vmax, vmax=vmax,
        legend_kwds={
            'label': "Relative Growth Rate",
            'orientation': "horizontal",
        }
    )

    # Set title
    years = df_pivot.index
    if title_var is None:
        title_var = f'Relative Growth of {variable} ({years.min()} - {years.max()})'
    ax.set_title(title_var, fontsize=20)

    # Change font sizes
    cbar = fig.axes[-1]
    cbar.set_xlabel('Relative Growth Rate', fontsize=font_size)
    for label in cbar.get_xticklabels():
        label.set_fontsize(15)

    # Create a custom legend patch for "No Data"
    no_data_patch = mpatches.Patch(color='darkgrey', label='No Data')
    plt.legend(handles=[no_data_patch], loc='upper right', fontsize=font_size)

    # Remove axis
    plt.axis('off')

    # Save figure
    plt.show()
    save_fig(fig, f'growth_rate_{variable.replace(" ", "_")}', 'water_management', experimental=True)

In [ ]:
plot_growth_rates(
    waste_df,
    'Number of municipal wastewater treatment facilities per 1000 citizens',
    'coolwarm_r',
    'Municipal Wastewater Treatment Facilities Per 1000 Citizens'
)

In [ ]:
plot_growth_rates(
    waste_df,
    'Number of municipal wastewater treatment facilities',
    'coolwarm_r',
    'Total Number of Municipal Wastewater Treatment Facilities'
)

It's odd to see that so many countries have a negative (redish) or a stagnant (dark grey) growth rate which means they stopped building facilities in the last years. Is there a reason for that? Maybe the water stress did not increase in these countries?

Water stress (SDG 6.4.2) is defined as the ratio of total water withdrawals to available renewable freshwater resources.

In [ ]:
# Filter interesting variables
filter_for = ['stress']

filtered_columns = [column for column in aquastat_df.columns if any([x in column.lower() for x in filter_for])]
filtered_columns

In [ ]:
# Plot the water quality for each country
# The countries that have a negative slope and a bad water quality, are the ones we are interested in

# Create a new dataframe with only the columns we are interested in
interesting_var = 'SDG 6.4.2. Water Stress'

stress_df = aquastat_df[['Country', 'Year', interesting_var]]
stress_df = stress_df.dropna()  # Drop rows with missing values
stress_df = stress_df.sort_values(by=interesting_var, ascending=False)  # Sort by number of facilities

stress_df

In [ ]:
'''Just for testing'''
sns.scatterplot(data=stress_df[stress_df['Country'] == 'Mexico'], x='Year',
                y='SDG 6.4.2. Water Stress')
plt.show()

In [ ]:
plot_growth_rates(
    stress_df,
    'SDG 6.4.2. Water Stress',
    'coolwarm',
    'Water Stress'
)


it could be also interesting to look how their wastewater output has changed.

In [ ]:
# Create a new dataframe with only the columns we are interested in
interesting_var = 'Produced municipal wastewater'

wwater_df = aquastat_df[['Country', 'Year', interesting_var]]
wwater_df = wwater_df.dropna()  # Drop rows with missing values
wwater_df = wwater_df.sort_values(by=interesting_var, ascending=False)  # Sort by number of facilities

wwater_df

In [ ]:
plot_growth_rates(
    wwater_df,
    interesting_var,
    'coolwarm',
    'Produced Municipal Wastewater'
)

What we see is that most countries don't face a change in produced waste water.

# TODOS

- Plot the development of the number of facilities over time
- Plot the number of facilities per capita
- [HydroWASTE](https://www.hydrosheds.org/products/hydrowaste):
    - Download the data
    - Explore the data:
        - A nice map of the world
        - The more facilities, the more water is treated?
            - Plot the number of facilities against the water withdrawal
            - What about the countries that face water scarcity?
                - Plot the number of facilities against the water scarcity: facility / water scarcity -> should be normalized by water stress (water scarcity / water availability)  